In [ ]:
# Launching necessary AWS Services for End-to-End Communication using Amazon’s Rest API gateway

In [ ]:
'''
Use a local client (i.e., Postman in our case) to send a sample test data to the deployed model in the cloud, 
and get the prediction back to the client - Amazon Restful http methods.
'''

In [ ]:
'''
After completing the above steps, we will have the model deployed and a SageMaker endpoint ready to be invoked 
from outside world to get real time predictions from the deployed model using the channel as:-
-----------------------------------------------------------------------------------------------------------------
 client (web browser) -> Amazon API Gateway -> Lambda (Call runtime: invoke endpoint) -> SageMaker Model endpoint
 -----------------------------------------------------------------------------------------------------------------

A client script calls an Amazon API Gateway API action and passes parameter values. 
API Gateway is a layer that provides the API to the client. API Gateway passes the parameter values 
to the Lambda function. 

The Lambda function parses the value and invokes the SageMaker model endpoint, passing the parameters to the same. 
The model performs the prediction task and returns the prediction results to Lambda. 
The Lambda function parses the returned value and sends it back to API Gateway. API Gateway responds to the client with that value.

We will use Amazon’s Rest API gateway for our purpose. 
Instead of a web browser as the client, we will use Postman app to keep things simple.

(If you want to use a browser web interface, you need Flask to be packed in a container 
which needs to be placed and run inside SageMaker). 
In our example, Postman will be used to send Restful POST method to call the API gateway and get the response 
(Predictions) back. 
So, we need to set up API gateway and Lambda.
'''

# Select Lambda as the use case in AWS service, while creating the role and attach the policy to the role.
#  Create a IAM role that includes the following policy, 
#  which gives your Lambda function permission to invoke a model endpoint.

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "sagemaker:InvokeEndpoint",
            "Resource": "*"
        }
    ]
}

In [ ]:
## STEP A) CREATE LAMBDA FUNCTION
           ----------------------

# Create a Lambda function with the below mentioned python code, that calls the SageMaker runtime invoke_endpoint 
# and returns the prediction.

# Select “Author from Scratch” and give a function name and select Runtime as Python 3.8 as shown below.

# Select “Use an existing role” and pick the role you created in the previous step.

# Under the code section of the lambda, enter the python code given below. Remember to click “Deploy” after entering the code.

# Go to the Configuration tab of the Lambda function and add an environment variable “ENDPOINT_NAME”
# and set it’s value as the same endpoint that was created in the preceding steps.
# Note that this environment variable is used in the Lambda function’s code.

# We have completed setting up the Lambda function.


import os
import boto3
import json
# grab environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']
runtime= boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    
    data = json.loads(json.dumps(event))
    payload = data['data']
    #print(payload)
    
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='text/csv',
                                       Body=payload)
    #print(response)
    result = json.loads(response['Body'].read().decode("utf-8"))

    res_list = [math.ceil(float(i)) for i in result]

    return result[0]

    label = payload.strip(" ").split()[0]
    print(f"Label: {label}\nPrediction: {result[0]}")

In [ ]:
## STEP B) Create a REST API and integrate with the Lambda function
           --------------------------------------------------------

# Create a REST API and integrate with the Lambda function

# Click on Build and select “New API”. In the next window you get, select “Create Resource” 
# from Actions drop-down menu, and enter a Resource Name.

# Note down the Resource Name you choose. It will be a part of the URL created by this service and will be used 
# later when we test the deployment from Postman. Here, we have chosen resource name as “housepriceprediction”. 
# After creating the resource, select “Create Method” from Actions drop-down menu.

# Select POST method and “Lambda Function” as Integration type. 
# Enter the name of the Lambda Function you created in the previous steps. 
# Then, select “Deploy API” from Actions drop-down menu. 
# Select Deployment stage “New Stage” and give some stage name. I chose to enter “test”.

# Then, finally when you click “Deploy”, you will be given a “Invoke URL” as shown below.

# Please note down the URL displayed on the window as “Invoke URL”. 
# It will be used in Postman to contact the API gateway, as described below.
# Now we are done with the deployment and setup of the end-to-end communication path.

In [ ]:
## STEP C) Testing the final Deployment from local client
           ----------------------------------------------

'''
Finally, use Postman App in your laptop, to POST the House price test data to API gateway and get the prediction result back from AWS cloud.
Example URL: (Remember to replace with the API URL you got when you created the API in the preceding steps, and append the resource name at the end.)

For example, if the Invoke URL you got was
https://kmnia554df.execute-api.us-east-1.amazonaws.com/test/
append the resource name to the above URL and use in the postman. For example in our case it was “housepriceprediction” . 

You can see the screen snapshot given below for the full example URL.
Use method : POST

In the Body, raw input can be given like:-

{"data":"3.77E-02, 8.00E+01, 1.52E+00, 0.00E+00, 4.04E-01, 7.27E+00, 3.83E+01, 
7.31E+00, 2.00E+00, 3.29E+02, 1.26E+01, 3.92E+02, 6.62E+00"}

You can refer to the "test_point_regression_sample.csv" for sample data. 

The thirteen numerical parameters given as data are nothing but features such as crim - per capita crime rate by town, zn - proportion of residential land zoned for lots over 25,000 sq.ft., indus - proportion of non-retail business acres per town,
chas - Charles River dummy variable (= 1 if tract bounds river; 0 otherwise), nox - nitrogen oxides concentration (parts per 10 million),
rm - average number of rooms per dwelling, age - proportion of owner-occupied units built prior to 1940, dis - weighted mean of distances to five Boston employment centres, 
rad - index of accessibility to radial highways, tax - full-value property-tax rate per $10,000, ptratio  pupil-teacher ratio by town, 
black - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town, lstat - lower status of the population (percent) of the Boston House Price data point.

# When we send the data, we successfully invoke the deployed model endpoint and receive back the house price prediction as “28.42” in the example above.

# So, we have successfully deployed a locally trained model on the AWS cloud using SageMaker and seen it working for real-time inference !
'''